In [1]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
# display(HTML("<style>.container { font-weight: bold !important; font-family:'Malgun Gothic' !important;}</style>"))
display(HTML("<style>.container { font-weight: bold !important;}</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

- 캐글 사이트에서 로그인후 다운로드 
https://www.kaggle.com/uciml/pima-indians-diabetes-database#

1) diabetes.csv 파일을 이용하여 데이타프레임을 생성하여라.<br> 

2) 결측값이 있는지 확인하여라.<br> 

3) 각 피처별 최소값이 0으로 측정되는 데이타값을 평균값으로 변경하여라.<br> 

4) StandardScaler()를 이용하여 스케일링하여라.<br> 

5) 학습 데이타셋과 테스트 데이타셋으로 분리시키고 모델을 생성한 후 
   머신러닝 예측 모델을 수립하고 학습하여라 


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                            random_state = 156, stratify=y)
    model_dt = DecisionTreeClassifier(random_state=11)


6) 오차행렬, 재현율, 정밀도, 정확도, F1 Score 등의 평가지표를 출력하여라 

7) 결정트리 그래프 시각화 

8) 아래의 소스를 참조하여 그리드서치를 적용시켜 최적의 하이퍼파라미터 예측 모델을 생성하여라. 

    model_dt = DecisionTreeClassifier(random_state=11)

    parameters = {'max_depth':[2,3,5,10] }

    model_dt_grid = GridSearchCV(model_dt , param_grid=parameters , scoring='accuracy' , cv=5)

9) 최적의 하이퍼파라미터로 모델을 생성하고 다시 학습시킨 후 평가지표를 확인하여라 

10) 가장 중요도가 높은 피처 2개는 무엇인가?


<!-- 아래그림 확인 -->
<img src="https://thebook.io/img/080228/137.jpg">
<br>
<img src="http://health.chosun.com/site/data/img_dir/2005/09/29/c2005092956311_01.jpg">

* Pregnancies: 임신 횟수
* Glucose: 포도당 부하 검사 수치
* BloodPressure: 혈압(mm Hg)
* SkinThickness: 팔 삼두근 뒤쪽의 피하지방 측정값(mm)
* Insulin: 혈청 인슐린(mu U/ml)
* BMI: 체질량지수(체중(kg)/(키(m))^2)
* DiabetesPedigreeFunction: 당뇨 내력 가중치 값
* Age: 나이
* Outcome: 클래스 결정 값(0 또는 1)

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import cross_val_score 
# from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.metrics import classification_report



In [11]:
ls data\d*.csv

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: E070-A7AC

 C:\jupyterworkspace\PythonLibrary\data 디렉터리

2021-12-07  오후 04:12            23,873 diabetes.csv
               1개 파일              23,873 바이트
               0개 디렉터리  440,106,901,504 바이트 남음


In [25]:
diabetes_data = pd.read_csv('data/diabetes.csv')
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [26]:
diabetes_data.shape

(768, 9)

In [27]:
# Outcome : 클레스 결정 값(0또는 1)
# 0 => 정상, 1 => 당뇨병
# 당뇨병 268명 확인
diabetes_data['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [28]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [36]:
# 피처 데이터 세트 X, 레이블 데이터 세트 y를 추출
# 맨 끝이 Outcome 컬럼으로 레이블 값임, 컬럼 위치 -1을 이용해 추출
X = diabetes_data.iloc[:, :-1]
y = diabetes_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, random_state=11, stratify=y)

# 로지스틱 회귀로 학습, 예측 및 평가 수행
lr_clf = LogisticRegression()
lr_clf.fit(X_train,y_train)
pred = lr_clf.predict(X_test)
pred_proba = lr_clf.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, pred, pred_proba)

NameError: name 'get_clf_eval' is not defined